In [18]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

def cust_lemma(tokens):
    for i in tokens:
        if len(i)<3 and i.isdigit()==False:
            tokens.remove(i)   

words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)


for intent in intents['intents']:
    for pattern in intent['patterns']:

        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# lemmaztize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]

#remove two letter words
cust_lemma(words)
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)


pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")


# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=300, batch_size=5, verbose=1)
model.save('model.h5', hist)

print("model created")


73 documents
13 classes ['Abdurrahman', 'Abu Bakr 1', 'Abu Bakr 2', 'Ali', 'Ali 2', 'General', 'Sad Waqqaas', 'Umar 1', 'Umar 2', 'Uthman', 'Zubair', 'juma', 'maghfirat']
60 unique lemmatized words ['1', '10', '11', '2', '3', '4', '5', '6', '7', '8', '9', 'abdurrahman', 'abo', 'abu', 'ali', 'allah', 'asbaab', 'ashra', 'aur', 'azeem', 'bakr', 'bhoolein', 'bhoolo', 'biography', 'din', 'dua', 'dusra', 'first', 'foregiveness', 'ijtemai', 'isteghfaar', 'istighfaar', 'juma', 'karna', 'key', 'kirdaar', 'lamha', 'life', 'maango', 'maghfirat', 'mubashira', 'part', 'pehla', 'pucho', 'qabooliyat', 'rabb', 'rub', 'saat', 'sad', 'second', 'seerah', 'seerat', 'sey', 'taalib', 'tadbeer', 'umar', 'uthman', 'waqqaas', 'zindagi', 'zubair']
Training data created
Epoch 1/300
73/73 [==============================] - 0s 1ms/step - loss: 2.5883 - accuracy: 0.1233
Epoch 2/300
73/73 [==============================] - 0s 274us/step - loss: 2.4929 - accuracy: 0.1507
Epoch 3/300
73/73 [===========================

73/73 [==============================] - 0s 233us/step - loss: 0.1074 - accuracy: 0.9726
Epoch 74/300
73/73 [==============================] - 0s 233us/step - loss: 0.1321 - accuracy: 0.9863
Epoch 75/300
73/73 [==============================] - 0s 219us/step - loss: 0.0910 - accuracy: 0.9863
Epoch 76/300
73/73 [==============================] - 0s 233us/step - loss: 0.0931 - accuracy: 0.9863
Epoch 77/300
73/73 [==============================] - 0s 247us/step - loss: 0.1148 - accuracy: 0.9726
Epoch 78/300
73/73 [==============================] - 0s 247us/step - loss: 0.1263 - accuracy: 0.9589
Epoch 79/300
73/73 [==============================] - 0s 260us/step - loss: 0.1232 - accuracy: 0.9589
Epoch 80/300
73/73 [==============================] - 0s 233us/step - loss: 0.1269 - accuracy: 0.9726
Epoch 81/300
73/73 [==============================] - 0s 247us/step - loss: 0.1478 - accuracy: 0.9452
Epoch 82/300
73/73 [==============================] - 0s 356us/step - loss: 0.0872 - accuracy: 

73/73 [==============================] - 0s 219us/step - loss: 0.0719 - accuracy: 0.9726
Epoch 153/300
73/73 [==============================] - 0s 233us/step - loss: 0.0548 - accuracy: 0.9726
Epoch 154/300
73/73 [==============================] - 0s 219us/step - loss: 0.0515 - accuracy: 1.0000
Epoch 155/300
73/73 [==============================] - 0s 251us/step - loss: 0.0325 - accuracy: 1.0000
Epoch 156/300
73/73 [==============================] - 0s 216us/step - loss: 0.0417 - accuracy: 0.9863
Epoch 157/300
73/73 [==============================] - 0s 228us/step - loss: 0.0263 - accuracy: 0.9863
Epoch 158/300
73/73 [==============================] - 0s 224us/step - loss: 0.0728 - accuracy: 0.9726
Epoch 159/300
73/73 [==============================] - 0s 228us/step - loss: 0.0711 - accuracy: 0.9726
Epoch 160/300
73/73 [==============================] - 0s 233us/step - loss: 0.0746 - accuracy: 0.9863
Epoch 161/300
73/73 [==============================] - 0s 238us/step - loss: 0.0440 - a

73/73 [==============================] - 0s 233us/step - loss: 0.0295 - accuracy: 1.0000
Epoch 232/300
73/73 [==============================] - 0s 219us/step - loss: 0.0267 - accuracy: 1.0000
Epoch 233/300
73/73 [==============================] - 0s 219us/step - loss: 0.0131 - accuracy: 1.0000
Epoch 234/300
73/73 [==============================] - 0s 233us/step - loss: 0.0422 - accuracy: 1.0000
Epoch 235/300
73/73 [==============================] - 0s 233us/step - loss: 0.0118 - accuracy: 1.0000
Epoch 236/300
73/73 [==============================] - 0s 260us/step - loss: 0.0276 - accuracy: 1.0000
Epoch 237/300
73/73 [==============================] - 0s 233us/step - loss: 0.0960 - accuracy: 0.9589
Epoch 238/300
73/73 [==============================] - 0s 233us/step - loss: 0.0264 - accuracy: 0.9863
Epoch 239/300
73/73 [==============================] - 0s 219us/step - loss: 0.0566 - accuracy: 1.0000
Epoch 240/300
73/73 [==============================] - 0s 233us/step - loss: 0.0218 - a